In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import java.sql.Date
import scala.collection.JavaConverters._

import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import java.sql.Date
import scala.collection.JavaConverters._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Hello Spark")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
23/10/06 18:35:29 INFO SparkContext: Running Spark version 3.4.1
23/10/06 18:35:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/06 18:35:29 INFO ResourceUtils: ==============================================================
23/10/06 18:35:29 INFO ResourceUtils: No custom resources configured for spark.driver.
23/10/06 18:35:29 INFO ResourceUtils: ==============================================================
23/10/06 18:35:29 INFO SparkContext: Submitted application: Hello Spark
23/10/06 18:35:29 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
23/10/06 18:35:29 INFO Resource

spark: SparkSession = org.apache.spark.sql.SparkSession@351f67c8
import spark.implicits._

In [4]:
println(s"spark.version == ${spark.version}")
val sc= spark.sparkContext

spark.version == 3.4.1


sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@593f6c53

In [5]:
case class Person(id: Int, name: String, age: Int)
val rdd = sc.parallelize(List(Person(1, "Name1", 20), Person(2, "Name2", 30), Person(3, "Name3", 40)))

defined class Person
rdd: org.apache.spark.rdd.RDD[Person] = ParallelCollectionRDD[0] at parallelize at cell5.sc:2

In [6]:
val df = rdd.toDF

23/10/06 18:36:46 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/10/06 18:36:46 INFO SharedState: Warehouse path is 'file:/Users/vadim/Projects/otus/Spark/HelloSpark/notebooks/spark-warehouse'.


org.apache.spark.sql.AnalysisException: Unable to generate an encoder for inner class `ammonite.$sess.cell5$Helper$Person` without access to the scope that this class was defined in.
Try moving this class out of its parent class.

In [6]:
df.printSchema

cell7.sc:1: not found: value df
val res7 = df.printSchema
           ^
Compilation Failed

In [ ]:
df.show

# DataFrame

In [7]:
val df = spark.createDataFrame(Array(
    (1, 2.0, "string1", Date.valueOf("2000-01-01")),
    (2, 3.0, "string2", Date.valueOf("2000-02-01")),
    (3, 4.0, "string3", Date.valueOf("2000-03-01"))
))
df

df: DataFrame = [_1: int, _2: double ... 2 more fields]
res7_1: DataFrame = [_1: int, _2: double ... 2 more fields]

In [8]:
df.printSchema

root
 |-- _1: integer (nullable = false)
 |-- _2: double (nullable = false)
 |-- _3: string (nullable = true)
 |-- _4: date (nullable = true)



In [9]:
val schema = StructType(
    StructField("a", IntegerType, nullable = true) ::
    StructField("b", DoubleType, nullable = true) ::
    StructField("c", StringType, nullable = true) ::
    StructField("d", DateType, nullable = true) ::
    Nil)

schema: StructType = StructType(
  StructField("a", IntegerType, true, {}),
  StructField("b", DoubleType, true, {}),
  StructField("c", StringType, true, {}),
  StructField("d", DateType, true, {})
)

In [10]:
val sc = spark.sparkContext
val df1 = spark.createDataFrame(sc.parallelize(
    Seq(
        Row(1, 2.0, "string1", Date.valueOf("2000-01-01")),
        Row(2, 3.0, "string2", Date.valueOf("2000-02-01")),
        Row(3, 4.0, "string3", Date.valueOf("2000-03-01"))
    )),
    schema)
df1

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@593f6c53
df1: DataFrame = [a: int, b: double ... 2 more fields]
res10_2: DataFrame = [a: int, b: double ... 2 more fields]

In [11]:
df1.printSchema

root
 |-- a: integer (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)



In [12]:
df1.dtypes

res12: Array[(String, String)] = Array(
  ("a", "IntegerType"),
  ("b", "DoubleType"),
  ("c", "StringType"),
  ("d", "DateType")
)

In [ ]:
df1.show()

In [ ]:
df1.columns

In [ ]:
df1.select("a", "b", "c").describe().show()

In [ ]:
df1.withColumn("upper_c", upper($"c")).show()

In [ ]:
df1.filter($"a" === 1).show()

In [ ]:
val df2 = spark.createDataFrame(List(
    Row("red", "banana", 1, 10),
    Row("blue", "banana", 2, 20),
    Row("red", "carrot", 3, 30),
    Row("blue", "grape", 4, 40),
    Row("red", "carrot", 5, 50),
    Row("black", "carrot", 6, 60),
    Row("red", "banana", 7, 70),
    Row("red", "grape", 8, 80)).asJava,
StructType(Array(
    StructField("color", StringType, nullable = true),
    StructField("fruit", StringType, nullable = true),
    StructField("v1", IntegerType, nullable = true),
    StructField("v2", IntegerType, nullable = true)))
)

df2.show()

In [ ]:
df2.groupBy("color").avg().show()

In [ ]:
df2.write.mode("overwrite").option("header", "true").csv("foo.csv")
spark.read.option("header", "true").csv("foo.csv").show()

In [ ]:
df.write.mode("overwrite").parquet("bar.parquet")
spark.read.parquet("bar.parquet").show()

In [ ]:
df.write.mode("overwrite").orc("zoo.orc")
spark.read.orc("zoo.orc").show()

In [ ]:
df2.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
val add_one = (a: String) => a + 1
val add_oneUDF = udf(add_one)

df2.select(add_oneUDF($"v1").as("add_one_v1")).show

In [ ]:
spark.udf.register("add_one", add_oneUDF)
spark.sql("SELECT add_one(v1) FROM tableA").show()